In [6]:
import numpy as np                #Import the necessary packages
import cv2
import matplotlib.pyplot as plt
import os
import csv

In [7]:
def extractor(img, label_img, output_path):
    
    '''
    Função que extrai os atributos, tais atributos foram calculados pelo próprio Opencv em metódos já implementados

    foram extraídos 11 atributos: 
    1 - média da vizinhança menor
    2 - máximo da vizinhança menor
    3 - mínimo da vizinhança menor
    4 - desvio padrão da vizinhança menor
    5 - valor do pixel central
    6 - desvio padrão local (média local - valor do pixel central) da vizinhança menor
    7 - média da vizinhança maior
    8 - máximo da vizinhança maior
    9 - mínimo da vizinhança maior
    10 - desvio padrão da vizinhança maior
    11 - desvio padrão local (média local - valor do pixel central) da vizinhança maior
    12 - Classe do pixel de acordo com o label feito manualmente

    ** Implementada por mim **

    Foram extraídas informações de duas vizinhanças diferentes, umas de tamanho 9x9 e outra de tamanho 27x27, dessa forma
    é consigo extrair caracteristicas locais e regionais a respeito do pixel
    ''' 
    
    (h, w) = img.shape[:2]  
    roi_sizes = [4,13]        
    result = []                        
    val_vector = []
    
    for y in range(0,h):
        for x in range(0,w):  
                                      
            val_vector
            roi1 = roi_treatment(img, x, y, roi_sizes[0])
            roi2 = roi_treatment(img, x, y, roi_sizes[1]) 
            
            val_vector.append(roi1.mean())                    
            val_vector.append(roi1.max())       
            val_vector.append(roi1.min())                    
            val_vector.append(roi1.std())
            val_vector.append(img[y,x])
            val_vector.append(roi1.mean() - img[y,x])
            
            val_vector.append(roi2.mean())
            val_vector.append(roi2.max())
            val_vector.append(roi2.min())
            val_vector.append(roi2.std())
            val_vector.append(roi2.mean() - img[y,x])

            if label_img[y,x] == 255:
                label = 1
            elif label_img[y,x] == 0:
                label = 0
            else:
                print('The value ', label_img[y,x],' is not accepted, the label image should be binary')
            
            val_vector.append(label)            
            csv_writer(val_vector, output_path)                  
           
                
    return 0

In [8]:
'''
Função que retorna as regiões de interesse, já com o tratamento das bordas

** Implementada por mim **
'''


def roi_treatment(img, x, y, roi_size):
    
    '''
    This is a function to treat the border pixels, only inside image pixels are considered as neighbors
    '''
    
    w = img.shape[1]
    h = img.shape[0]
    
    if(x-roi_size <= 0):
        x1 = 0
    else:
        x1 = x-roi_size
        
    if(x+roi_size >= w-1):
        x2 = w-1
    else:
        x2 = x+roi_size
    
    
    if(y-roi_size <= 0):
        y1 = 0
    else:
        y1 = y-roi_size
        
    if(y+roi_size >= h-1):
        y2 = h-1
    else:
        y2 = y+roi_size
        
    roi = img[y1:y2+1, x1:x2+1]    
    
    return roi
    

In [9]:
'''
Função que salva os resultados em um csv

*** Implementada por mim ***
'''

def csv_writer(val_vector, output_path):
    
    key_vector = ['mean_roi1','max_roi1','min_roi1','std_roi1','center_pixel_value','loc_std_roi1',
                  'mean_roi2','max_roi2','min_roi2','std_roi2','loc_std_roi2', 'Label']
        
    if not os.path.exists(output_path):
        with open(output_path, 'w+') as f:
            for i in range(len(key_vector)):
                aux_escrita = str(i)+'_'+str(key_vector[i])
                f.write(str(aux_escrita))
                if i == len(key_vector)-1:
                    #f.write('\t')
                    #f.write('Class')
                    f.write('\n')
                else:
                    f.write('\t')
            for i in range(len(val_vector)):
                f.write(str(val_vector[i]))
                if i == len(val_vector)-1:
                    f.write('\n')
                else:
                    f.write('\t')
    else:
        with open(output_path, 'a') as f:
            for i in range(len(val_vector)):
                f.write(str(val_vector[i]))
                if i == len(val_vector)-1:
                    f.write('\n')
                else:
                    f.write('\t')

In [10]:
'''
input_path - onde se encontram as imagens a serem binarizadas
labels_path - onde se encontram as imagens de label feitas manualmente
output_path - onde será salvo o csv
csv_file_name - nome do arquivo csv

'''
if __name__ == "__main__":

    input_path = r'~\DocumentCleanup\data\01-DocumentCleanup'
    labels_path = r'~\DocumentCleanup\data\Labels'
    output_path = r'~\DocumentCleanup\data'
    csv_file_name = 'training_features.csv'

    images = os.listdir(labels_path)
    
    for im in images:
        
        img = cv2.imread(input_path+'\\'+im,0) 
        label_img = cv2.imread(labels_path+'\\'+im,0)

        extractor(img, label_img, output_path+'\\'+csv_file_name)
